In [1]:
import faiss

https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6

In [2]:
import pandas as pd
import numpy as np
from time import time
from tqdm import tqdm
tqdm.pandas()

model_name = 'seresnext101'
fold = 2
checkpoint = '20600'
algo = 'dist_global_org'

nums = [model_name,fold,
        checkpoint,
        model_name,
        fold,algo]

In [3]:
train = pd.read_csv('../WC_input/train.csv')

In [4]:
train.set_index('Image').loc['PM-WWA-20140813-067.jpg']

Id    -1
Name: PM-WWA-20140813-067.jpg, dtype: object

In [5]:
dic = train.set_index('Image').to_dict(orient='dict')['Id']

In [6]:
top20 = pd.read_csv('../WC_result/{0}_{1}/out_{2}/{3}_sub_fold{4}_{5}.csv'.format(*nums), header=None)
top20 = top20.set_index(0)

In [15]:
enc = pd.read_csv('../WC_result/{0}_{1}/out_{2}/encoding_org_img.csv'.format(*nums), header=None)
enc.shape

(5347, 2049)

In [7]:

enc = enc.set_index(0)

enc['embeddings'] = enc.values.tolist()

enc = enc.reset_index()

enc = enc.iloc[:, [0, 2050-1]]

enc.columns = ['Image', 'embeddings']

enc.head(1)

,Image,embeddings
0,PM-WWA-20180811-093.jpg,"[0.030311882000000002, 0.0005835553, -0.031396..."


In [8]:
train.shape

(4539, 2)

In [9]:
enc = enc.merge(train, on='Image', how='left')

In [10]:
enc = enc[enc.Id!='-1']

In [11]:
enc = enc.reset_index(drop=True)

In [14]:
enc.shape

(4770, 3)

In [12]:
sdfsgd

NameError: name 'sdfsgd' is not defined

## Exhaustive Search Usage


In [ ]:
USE_GPU = True

class ExactIndex():
    def __init__(self, vectors, labels):
        self.dimension = vectors.shape[1]
        self.vectors = np.ascontiguousarray(vectors.astype('float32'))
        self.labels = labels    
   
    def build(self):
        self.index = faiss.IndexFlatL2(self.dimension,)
#         if USE_GPU:
#             res = faiss.StandardGpuResources()
#             self.index = faiss.index_cpu_to_gpu(res, 0, self.index)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return distances, [self.labels[i] for i in indices[0]]

In [ ]:
index = ExactIndex(enc["embeddings"].apply(pd.Series).values, enc["Image"].values)
index.build()

In [ ]:
enc["Image"].values

In [ ]:
result = np.ascontiguousarray(enc["embeddings"].apply(pd.Series).values).astype('float32')

In [ ]:
result[0]

In [ ]:
result[1]

In [ ]:
enc.head(1)

In [ ]:
test = enc[enc.Image.isin(top20.index)]
test = test.reset_index() 
test.columns = ['id', 'Image' , 'embeddings', 'class']

In [ ]:
test.shape

In [ ]:
test

In [ ]:
test[test.Image.isin(top20.index)].shape

In [ ]:
test

In [ ]:
dat = np.ascontiguousarray(enc["embeddings"].apply(pd.Series).values).astype('float32')

In [ ]:
test['top20dist'] = np.nan
test['top20imgs'] = np.nan

In [ ]:
test['top20imgs'] = test['id'].map(lambda x: index.query(np.expand_dims(dat[x], 1).reshape(1, -1), 100))

In [ ]:
test['top20dist'] = test['top20imgs'].map(lambda x: x[0][0][1:])
test['top20imgs'] = test['top20imgs'].map(lambda x: x[1][1:])

In [ ]:
test['top20imgs'].head()

In [ ]:
test['top20dist'].head()

In [ ]:
fp = '/home/pt-support/Humpback-Whale-Identification-1st-/WC_input/'
train = pd.read_csv(fp+'train.csv') 
image_to_id = dict(zip(train.Image, train.Id))

In [ ]:
res = pd.concat([test['top20imgs'].apply(pd.Series), 
                 test['top20dist'].apply(pd.Series),
                 test['top20imgs'].apply(pd.Series).applymap(lambda x: image_to_id[x] if x in dic.keys() else np.nan)
                ], axis=1, keys=["img", "dist", "class"])
# res = res.swaplevel(0, 1, axis=1).sort_index(axis=1)

In [ ]:
confident_labels_test = pd.concat([res.img.loc[res['class'].loc[:, 0][(res.dist.loc[:, 0]<1)].dropna().index, 0],
           res['class'].loc[:, 0][(res.dist.loc[:, 0]<1)].dropna()], axis=1, keys=['Image','Id']).reset_index(drop=True)

In [ ]:
confident_labels_test.to_csv(fp + 'pseudo_labels.csv', index=False)

In [ ]:
jhgfdsa

In [ ]:
confident_labels_test.sample()

In [ ]:
pd.DataFrame(test.Image).join(test['top20imgs'].apply(pd.Series)).to_csv('faiss.csv', header=False, index=False)

In [ ]:
exp = test['top20imgs'].apply(pd.Series)

In [ ]:
test['top20imgs'].apply(pd.Series).applymap(lambda x: image_to_id[x] if x in dic.keys() else np.nan)

In [ ]:
exp_class = exp.applymap(lambda x: image_to_id[x] if x in dic.keys()else np.nan)

In [ ]:
res = pd.concat([exp, exp_class], axis=1, keys=["img", "class"])

In [ ]:
res = res.swaplevel(0, 1, axis=1).sort_index(axis=1)

In [ ]:
res.iloc[i].swaplevel(0, 1)['class'].dropna()[:20].index

In [ ]:
res.iloc[i].swaplevel(0, 1)['img'][res.iloc[i].swaplevel(0, 1)['class'].dropna()[:20].index]

In [ ]:
collector = []
for i in range(exp.shape[0]):
    collector.append(res.iloc[i].swaplevel(0, 1)['img'][res.iloc[i].swaplevel(0, 1)['class'].dropna()[:20].index].reset_index(drop=True))

In [ ]:
result = pd.DataFrame(test.Image).join(pd.concat(collector, axis=1, ignore_index=True).T)

In [ ]:
result.shape

In [ ]:
result.set_index('Image').loc['PM-WWA-20180811-093.jpg']

In [ ]:
result.to_csv('faiss_remove_test.csv', header=False, index=False)